In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import scipy.io as scio
import scipy.optimize as scop
import math
import cv2
import time
import random
from func_util_geom import *
from ctypes import *

In [3]:


def func_refine_pt3d_reprojection_sp_smoothing(fc, cc, R_l, tw_l, X, x_l, sp_weight=None):
    # If sp_weight is None, 3d points are independent 
    nosteps = len(R_l)
    nocams = len(R_l[0])
    X0 = np.concatenate(X, axis=0)

    def smoothermatrix():
        hi = np.ones((nosteps-2,),dtype=(float))
        hi[0]=2
        hi[-1]=2
        delta = np.zeros((nosteps-2,nosteps), dtype=float) 
        W = np.zeros((nosteps-2,nosteps-2), dtype=float) 

        for i in range(nosteps-2):
            delta[i,i] =  1 / hi[i]
            delta[i,i+1] =  - 1 / hi[i] - 1 / hi[i]
            delta[i,i+2] =  1 / hi[i]

        for i in range(nosteps-2):
            W[i,i] =  (hi[i]+hi[i])/3
            if (i>0):
                W[i-1,i] =  hi[i]/6
                W[i,i-1] =  hi[i]/6

        #print W
        #print delta
        K = np.dot(np.dot(delta.T, np.linalg.inv(W)), delta)
        #normfull = np.sum(K_filt,axis=0)

        return K

    if (sp_weight is not None):
        sm = smoothermatrix()
    
    def err_functional(Xin):
        err = np.zeros((2*nosteps*nocams + 3,), dtype=float)
        #err = np.zeros((2*nosteps*nocams + 3*nosteps,), dtype=float)
        for t in xrange(nosteps):
            for j in xrange(nocams):
                Xthis = Xin[(t*3):((t+1)*3),None].T
                x_hom = func_reproject(Xthis, R_l[t][j], tw_l[t][j], fc, cc)[0,[0,1]]
                #Xthis = np.concatenate((Xin[(t*3):((t+1)*3)],np.array([1.0])), axis=0)
                #x_hom = np.dot(P_l[t][j], Xthis)
                #x_hom = x_hom[[0,1]] / x_hom[2]
                #print x_hom, x_l[t][j][[0,1]]
                residual = x_hom - x_l[t][j][[0,1]]
                #print "Res: ", residual
                err[2*(j + t*nocams)  ] = residual[0]
                err[2*(j + t*nocams)+1] = residual[1]

        # Bending energy constraint for moving point    
        if (sp_weight is not None):
            Xin_reshape = Xin.reshape([-1,3]) 
            bendingerror= np.multiply(np.dot(Xin_reshape.T, sm), Xin_reshape.T)
            err[-3::] = sp_weight * np.sum(bendingerror, axis=1) # sum per dimension
            #err[-(3*nosteps)::] = sp_weight * np.abs(bendingerror).flatten()
            

        return err

    xres = scop.least_squares(err_functional, X0, method='lm', verbose=1).x
    pt_res = xres.reshape([-1,3]) 
    
    return pt_res



In [4]:

fc = np.array([2000,2000]).astype(float)
cc = np.array([0,0]).astype(float)
wh = np.array([1920,1080]).astype(float)


## NOTE: no preconditioning, coordinate normalization used here! Implement for better performance!


# Create cameras and 3d point
#pt = np.array(np.random.uniform((10,10,10)))
pt = np.array(np.random.uniform((15,15,15)))
#pt = np.array([15.0,15.0,15.0])
ptproj_GT = np.concatenate((cc,np.array([1.0])), axis=0) # 2d point position in the image, on optical center


# Create N random cameras with pt on Z axist of camera
R_l = []
tw_l = []
nocams = 3
for i in xrange(nocams):
    tw_l.append(np.random.uniform((10,10,10)))
    R3 = pt - tw_l[i]
    R3 /= np.linalg.norm(R3)
    R2 = np.cross(R3, np.random.uniform((10,10,10)))
    R2 /= np.linalg.norm(R2)
    R1 = np.cross(R3, R2)
    R_l.append(np.vstack((R2,R1,R3)))    
    
    #P_l.append(func_get_P_from_KRt(K,R_l[-1],tw_l[-1]))
    #ptproj = np.dot(P_l[-1], np.concatenate((pt,np.array([1.0])), axis=0))
    
    #ptproj = func_reproject(pt[:,None].T, R_l[i], tw_l[i], fc, cc)
    #print "reprojtest" , ptproj/ptproj[0,2]
    
    # SANITY CHECK: points have to lie exactly in image center
    #assert (np.all((ptproj[0,[0,1]] / ptproj[0,2]) - ptproj_GT[[0,1]] < 1e-5)) 

#pt = pt + 2.0
    
    
x_l = []
for i in xrange(nocams):
    #t_c = np.dot(-R_l[i], tw_l[i])[None,:]
    #t_c = np.dot(-R_l[i].T, tw_l[i])[None,:]
    #t_c = np.dot(R_l[i].T, tw_l[i])[None,:]
    #t_c = np.dot(R_l[i], tw_l[i])[None,:]
    ptproj = func_reproject(pt[:,None].T, R_l[i], tw_l[i].T, fc, cc, camcenter=0)[0,:]
    #ptproj = func_reproject(pt[:,None].T, R_l[i], t_c, fc, cc)[0,:]
    #print "A", ptproj
    
    #P_l = func_get_P_from_KRt(fc, cc, R_l[i],tw_l[i])
    #ptproj = np.dot(P_l, np.concatenate((pt,np.array([1.0])), axis=0))
    #ptproj /= ptproj[[2]]
    #print "B", ptproj
    
    
    #print ptproj/ptproj[[2]]    
    #x_l.append(ptproj[[1,0]]/ptproj[[2])
    x_l.append(ptproj)
    
    
pt_solv = func_pt_triangulate_from_P_linear_eigen(fc, cc, R_l, tw_l, x_l) 
pt_solv_ref = func_refine_pt3d_reprojection_sp_smoothing(fc, cc, [R_l], [tw_l], [pt_solv+0.1], [x_l])

#def func_pt_triangulate_from_P_depth_only(fc, cc, R_l, tw_l, x_l):
#    return X, XtX


#print x_l


print str(pt_solv) + " Sol.Homog. pos: "
print str(pt_solv_ref) + " Sol.Homog. pos. ref: "
print str(func_pt_triangulate_from_P_linear_sq(fc, cc, R_l, tw_l, x_l)[0]) + " Sol.Inhomog. pos: "
pt_sol_refdepth = func_pt_triangulate_from_P_nonlin_LM(pt_solv.copy()+1.1, fc, cc, R_l, tw_l, x_l, 10, 1, verbose=0)
print str(pt_sol_refdepth) + " Depth-only. pos with LM: "
print str(pt) + " GT pos: "






`xtol` termination condition is satisfied.
Function evaluations 13, initial cost 6.1800e+02, final cost 4.3400e-26, first-order optimality 6.51e-11.
[  5.38873718  13.07427209  12.51642292] Sol.Homog. pos: 
[[  5.38873718  13.07427209  12.51642292]] Sol.Homog. pos. ref: 


TypeError: 'NoneType' object has no attribute '__getitem__'

In [25]:
print x_l
print 

[array([ 0.,  0.]), array([  0.00000000e+00,  -9.28084494e-14]), array([ -1.43240759e-13,   0.00000000e+00])]
(2, 3)


In [ ]:
for i in xrange(3):
    for k in xrange(3):
        #for l in xrange(3):
            #print('%.15f' % R_l[i][k,l])
        print('%.15f' % tw_l[i][k])
    
    print " "

x_l

In [ ]:

# Triangulation covariance test

# Get covariance by sampling and triangulation
notrials = 5000
sig = 50.0

pt_rnd = []
for j in xrange(notrials):
    x_l = [ptproj_GT[[0,1]] + np.array([np.random.normal(0.0, sig), np.random.normal(0.0, sig)]) for i in xrange(nocams)]
    pt_rnd.append(func_pt_triangulate_from_P_linear_sq(fc, cc, R_l, tw_l, x_l)[0])

pt_rnd_cov = np.cov(np.stack(pt_rnd).T)
pt_rnd_mean = np.mean(np.stack(pt_rnd).T, axis=1)
ellsurf_samp = func_get_cov_ellipsoid(pt_rnd_cov, Nstd=3) + pt_rnd_mean[:,None,None]

# Get covariance from least squares triangulation, assumes noise on matrix A. 
# This is not exactly the same as propagating 2d observation noise through the triangulation. 
# This is an algebraic approximation to the true geometric noise. 
pt_solv, uncertainty_org = func_pt_triangulate_from_P_linear_sq(fc, cc, R_l, tw_l, [ptproj_GT] * nocams)

uncertainty = uncertainty_org*100000 # Scale it to be of same size for visual comparison. We only care about proportionality
ellsurf_lisq = func_get_cov_ellipsoid(uncertainty, Nstd=3) + pt_solv[:,None,None]

print "Ratio of determinants: ", np.linalg.det(pt_rnd_cov) / np.linalg.det(uncertainty)
    

    

# Plot ellsurf
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pt[0], pt[1], pt[2], color=np.array([1.0,0,0]), linewidth=2)
plt.hold('on')
ax.plot_surface(ellsurf_lisq[0,:], ellsurf_lisq[1,:], ellsurf_lisq[2,:],  rstride=4, cstride=4, color=[0.0,1.0,0.0], alpha=0.1)
ax.plot_surface(ellsurf_samp[0,:], ellsurf_samp[1,:], ellsurf_samp[2,:],  rstride=4, cstride=4, color=[0.0,0.0,1.0], alpha=0.1)

ax.scatter(pt_solv[0], pt_solv[1], pt_solv[2], color=np.array([0.0,1.0,0.0]), linewidth=2)
for i in xrange(nocams):
    col = np.array([1.0,0,0])
    if (i==0):
            col = np.array([0.0,0,0])
    func_plot_cameras(ax, fc, cc, wh, R_l[i], tw_l[i], rgbface=col, camerascaling=2)

func_set_axes_equal(ax)

plt.show()



In [ ]:
# Propagate 3D uncertainy into single image
pt = np.array([0,0,0])
ptproj_GT = np.concatenate((cc,np.array([1.0])), axis=0) # 2d point position in the image, on optical center


# Create N random cameras with pt on Z axist of camera
R_l = []
tw_l = []

tw_l = np.array([0.0,0.0,-10.0])
R3 = pt - tw_l
R3 /= np.linalg.norm(R3)
R2 = np.cross(R3, np.array([0.0,1.0,0.0]))
R2 /= np.linalg.norm(R2)
R1 = np.cross(R3, R2)
R_l = np.vstack((R2,R1,R3))
    

unc3d = np.diag([5.0,1.0,1.0])
ellsurf_lisq = func_get_cov_ellipsoid(unc3d, Nstd=1) + pt[:,None,None]

# Plot ellsurf
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pt[0], pt[1], pt[2], color=np.array([1.0,0,0]), linewidth=2)
plt.hold('on')
ax.plot_surface(ellsurf_lisq[0,:], ellsurf_lisq[1,:], ellsurf_lisq[2,:],  rstride=4, cstride=4, color=[0.0,1.0,0.0], alpha=0.1)
col = np.array([1.0,0,0])
func_plot_cameras(ax, fc, cc, wh, R_l, tw_l, rgbface=col, camerascaling=2)
func_set_axes_equal(ax)
plt.show()





plt.figure()
ax = plt.subplot(111)

pt2d = cc + wh/2.0

cov2d = func_propagate_project_3dcov_2dcov(unc3d, pt, fc, cc, R_l,tw_l)
print cov2d
plt.plot(pt2d[0], pt2d[1], 'ro')


nstd = 1
ell = func_get_cov_ellipe(cov2d, pt2d, nstd)[0]
ax.add_artist(ell)


plt.axis('image')
plt.axis([0, wh[0], wh[1], 0])

plt.show()

In [ ]:
nopt = 5
anchor1 = np.array(np.random.uniform((5,5,5))) + 1.0
anchor2 = np.array(np.random.uniform((5,5,5))) + 10.0
anchor3 = np.array(np.random.uniform((5,5,5))) + 1.0
pt = []
pt.append(np.stack((np.linspace(anchor1[0],anchor2[0],nopt),
                    np.linspace(anchor1[1],anchor2[1],nopt), 
                    np.linspace(anchor1[2],anchor2[2],nopt))))
pt.append(np.stack((np.linspace(anchor2[0],anchor3[0],nopt)[1::],
                    np.linspace(anchor2[1],anchor3[1],nopt)[1::], 
                    np.linspace(anchor2[2],anchor3[2],nopt)[1::])))
pt.append(np.stack((np.linspace(anchor3[0],anchor1[0],nopt)[1:-1],
                    np.linspace(anchor3[1],anchor1[1],nopt)[1:-1], 
                    np.linspace(anchor3[2],anchor1[2],nopt)[1:-1])))

pt = np.concatenate(pt, axis=1).T
        
nosteps = pt.shape[0]

# Create N random cameras with pt on Z axist of camera
R_l = []
tw_l = []
nocams = 3
for i in xrange(nocams):
    tw_l.append(np.random.uniform((10,10,10)))
    R3 = pt[0,:] - tw_l[-1]
    R3 /= np.linalg.norm(R3)
    R2 = np.cross(R3, np.random.uniform((10,10,10)))
    R2 /= np.linalg.norm(R2)
    R1 = np.cross(R3, R2)
    R_l.append(np.vstack((R2,R1,R3)))    
    
R_l = [R_l] * nosteps    
tw_l = [tw_l] * nosteps    

x_l = []
for t in xrange(nosteps):
    x_l.append([])
    for i in xrange(nocams):
        ptproj = func_reproject(pt[[t],:], R_l[t][i], tw_l[t][i].T, fc, cc)[0,:]
        x_l[t].append(ptproj)
    
#print np.stack(x_l[1], axis=0)

# Plot
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
plt.hold('on')

for t in xrange(nosteps):
    ax.scatter(pt[t,0], pt[t,1], pt[t,2], color=np.array([1.0,0,0]), linewidth=1)
    ax.plot(pt[:,0], pt[:,1], pt[:,2], color=np.array([1.0,0,0]), linewidth=1)
    
    pt_solv, uncertainty = func_pt_triangulate_from_P_linear_sq(fc, cc, R_l[t], tw_l[t], x_l[t])
    uncertainty = uncertainty*100000 # Scale it to be of same size for visual comparison. We only care about proportionality
    ellsurf_lisq = func_get_cov_ellipsoid(uncertainty, Nstd=3) + pt_solv[:,None,None]
    ax.plot_surface(ellsurf_lisq[0,:], ellsurf_lisq[1,:], ellsurf_lisq[2,:],  rstride=4, cstride=4, color=[0.0,1.0,0.0], alpha=0.1)

    
    # Get covariance by sampling and triangulation
    notrials = 500
    sig = 50.0

    pt_rnd = []
    for j in xrange(notrials):
        x_l_t = [x_l[t][i] + np.array([np.random.normal(0.0, sig), np.random.normal(0.0, sig)]) for i in xrange(nocams)]
        #print "DD" , x_l_t
        pt_rnd.append(func_pt_triangulate_from_P_linear_sq(fc, cc, R_l[t], tw_l[t], x_l_t)[0])

    pt_rnd_cov = np.cov(np.stack(pt_rnd).T)
    pt_rnd_mean = np.mean(np.stack(pt_rnd).T, axis=1)
    ellsurf_samp = func_get_cov_ellipsoid(pt_rnd_cov, Nstd=3) + pt_rnd_mean[:,None,None]
    ax.plot_surface(ellsurf_samp[0,:], ellsurf_samp[1,:], ellsurf_samp[2,:],  rstride=4, cstride=4, color=[0.0,0.0,1.0], alpha=0.1)


    #pt_solv = func_pt_triangulate_from_P_linear_eigen(fc, cc, R_l, tw_l,x_l[t])
    
    ax.scatter(pt_solv[0], pt_solv[1], pt_solv[2], color=np.array([0.0,1.0,0.0]), linewidth=2)
    
for i in xrange(nocams):
    func_plot_cameras(ax, fc, cc, wh, R_l[t][i], tw_l[t][i], rgbface=np.array([1.0,0,0]), camerascaling=2)
    
func_set_axes_equal(ax)

plt.show()


    


In [ ]:

pt_all = []
for t in xrange(nosteps):
    #pt_solv, uncertainty = func_pt_triangulate_from_P_linear_sq(fc, cc, R_l[t], tw_l[t], x_l[t])
    pt_solv, uncertainty = func_pt_triangulate_from_P_linear_sq(fc, cc, R_l[t], tw_l[t], [x_l[t][i] + np.random.uniform([5.5,5.5]) for i in xrange(nocams)])
    pt_all.append(pt_solv)

X = np.stack(pt_all)



spline_val = 2.0
weight = 300



pt_res = func_refine_pt3d_reprojection_sp_smoothing(fc, cc, R_l, tw_l, X, x_l, sp_weight=100)
    

print np.mean(np.linalg.norm(pt_res - pt, axis=1))
print np.mean(np.linalg.norm(X - pt, axis=1))

# Plot in 3D
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
plt.hold('on')    
for t in xrange(nosteps):
    ax.scatter(pt[t,0], pt[t,1], pt[t,2], color=np.array([1.0,0,0]), linewidth=1)
    ax.plot(pt[:,0], pt[:,1], pt[:,2], color=np.array([1.0,0,0]), linewidth=1)   
    ax.scatter(pt_res[t,0], pt_res[t,1], pt_res[t,2], color=np.array([0.0,0,1.0]), linewidth=1)
    ax.plot(pt_res[:,0], pt_res[:,1], pt_res[:,2], color=np.array([0.0,0,1.0]), linewidth=1)   

func_set_axes_equal(ax)
plt.show()


In [ ]:
# Plot in 2D


t=11 # plot only at time t=0
for i in xrange(nocams):
    plt.figure()
    pt2d = x_l[t][i] + wh/2.0
    pt2d_ref = func_reproject(pt_res, R_l[t][i], tw_l[t][i], fc, cc)[0,[0,1]] + wh/2.0
    print pt2d_ref
    plt.plot(pt2d[0], pt2d[1], 'ro')
    plt.hold('on')
    plt.plot(pt2d_ref[0], pt2d_ref[1], 'bo')
    plt.axis('image')
    plt.axis([0, wh[0], wh[1], 0])
    
    plt.show()